In [12]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

# Past Hammoz

In [8]:
model = xr.open_dataset('E://faks//cda//cmip6//ham_hist_interpo.nc')
time_slice = slice('1981-01-01T00:00:00.000000000','2014-12-31T23:30:00.000000000')
modelsel = model.sel(time=time_slice)
model_ham = modelsel.tas.drop_vars(['lat', 'lon','height']).sel(time=modelsel.time.dt.month.isin([1, 2, 3, 4, 10, 11, 12])) - 273.15

In [9]:
model_ham

<xarray.DataArray 'tas' (time: 173184)>
array([-2.201294 , -1.9090271, -1.6486816, ...,  7.2030334,  7.4024353,
        7.5882874], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01T00:29:59.999999488 ... 2014-12-...
    height   float64 ...
    lat      int32 ...
    lon      int32 ...

### Bias correction

In [14]:
bias_set = xr.open_dataset('E://faks//cda//era5//bias_monthly_7.nc')
bias = bias_set.__xarray_dataarray_variable__.values

In [15]:
bias

array([-0.66579727, -0.73899383,  0.91762081,  1.67808034, -0.05870329,
       -0.12518433, -0.38591374])

In [ ]:
model_ham_grouped = model_ham.groupby(model_ham.time.dt.month)
months = [1, 2, 3,4, 10, 11, 12]
bias_index = 0
for month in months:
    model_ham_without_bias = model_ham_grouped.apply(lambda x: x - bias[bias_index])
    bias_index = bias_index + 1

In [ ]:
model_ham_without_bias

## And now the apples..

### Utah

In [ ]:
temp=model_ham_without_bias
temp1=xr.where((temp>= 1.5) & (temp <= 2.4), 0.5 , np.nan)
temp2=xr.where((temp>= 2.5) & (temp <= 9.1), 1, np.nan)
temp3=xr.where((temp>= 9.2) & (temp <= 12.4), 0.5 , np.nan)
temp4=xr.where((temp>= 12.5) & (temp <= 15.9), 0 , np.nan)
temp5=xr.where((temp>= 16) & (temp <= 18), (-0.5) , np.nan)
temp6=xr.where((temp>= 18), -1 , np.nan)
hours = temp1 + temp2 + temp3 + temp4 + temp5 + temp6

In [ ]:
yearsum1 = temp1.groupby('time.year').sum(dim='time')
yearsum2 = temp2.groupby('time.year').sum(dim='time')
yearsum3 = temp3.groupby('time.year').sum(dim='time')
yearsum4 = temp4.groupby('time.year').sum(dim='time')
yearsum5 = temp5.groupby('time.year').sum(dim='time')
yearsum6 = temp6.groupby('time.year').sum(dim='time')

In [ ]:
yearsum = yearsum1 + yearsum2 + yearsum3 + yearsum4 + yearsum5 + yearsum6

In [ ]:
plt.plot(yearsum)

### Chill hour

In [ ]:
chill_hours = xr.where((temp>= 0) & (temp <= 7.2), 1 , np.nan)    #returns the variable 'chill_hours' with boolean values
chill_hours_year = chill_hours.groupby('time.year').sum(dim='time')
plt.plot(chill_hours_year, color='red')

In [ ]:
#comparison between apple models
plt.plot(chill_hours_year, color='red') #chill hour model
plt.plot(yearsum) #utah

# Future Hammoz

In [ ]:
model_ssp = xr.open_dataset('E://faks//cda//cmip6//Hammoz_ssp_merge_new.nc')
model_ham_ssp = model_ssp.tas.drop_vars(['lat', 'lon','height']).sel(time=model_ssp.time.dt.month.isin([1, 2, 3,4, 10, 11, 12])) - 273.15

### Bias correction

In [ ]:
model_ham_grouped_ssp = model_ham_ssp.groupby(model_ham_ssp.time.dt.month)
months = [1, 2, 3,4, 10, 11, 12]
bias_index = 0
for month in months:
    model_ham_without_bias_ssp = model_ham_grouped_ssp.apply(lambda x: x - bias[bias_index])
    bias_index = bias_index + 1

### Utah

In [ ]:
temp=model_ham_without_bias_ssp
temp1=xr.where((temp>= 1.5) & (temp <= 2.4), 0.5 , np.nan)
temp2=xr.where((temp>= 2.5) & (temp <= 9.1), 1, np.nan)
temp3=xr.where((temp>= 9.2) & (temp <= 12.4), 0.5 , np.nan)
temp4=xr.where((temp>= 12.5) & (temp <= 15.9), 0 , np.nan)
temp5=xr.where((temp>= 16) & (temp <= 18), (-0.5) , np.nan)
temp6=xr.where((temp>= 18), -1 , np.nan)
hours = temp1 + temp2 + temp3 + temp4 + temp5 + temp6

In [ ]:
yearsum1 = temp1.groupby('time.year').sum(dim='time')
yearsum2 = temp2.groupby('time.year').sum(dim='time')
yearsum3 = temp3.groupby('time.year').sum(dim='time')
yearsum4 = temp4.groupby('time.year').sum(dim='time')
yearsum5 = temp5.groupby('time.year').sum(dim='time')
yearsum6 = temp6.groupby('time.year').sum(dim='time')

In [ ]:
yearsum_ssp = yearsum1 + yearsum2 + yearsum3 + yearsum4 + yearsum5 + yearsum6

In [ ]:
plt.plot(yearsum_ssp)

### Chill hour

In [ ]:
chill_hours = xr.where((temp>= 0) & (temp <= 7.2), 1 , np.nan)    #returns the variable 'chill_hours' with boolean values
chill_hours_year_ssp = chill_hours.groupby('time.year').sum(dim='time')
plt.plot(chill_hours_year_ssp, color='red')

In [ ]:
#comparison between apple models
plt.plot(chill_hours_year_ssp, color='red') #chill hour model
plt.plot(yearsum_ssp) #utah

--------

------

-----------

Visualisations we decided on:

# Utah model

In [ ]:
#deviding the future into two periods:
time_slice1 = slice('2015-01-01T00:00:00.000000000','2035-12-31T23:30:00.000000000')
time_slice2 = slice('2035-01-01T00:00:00.000000000','2055-12-31T23:30:00.000000000')
yearsum_ssp1  = yearsum_ssp.sel(year=time_slice1).mean('year')
yearsum_ssp2  = yearsum_ssp.sel(year=time_slice2).mean('year')
chill_hours_year_ssp1  = chill_hours_year_ssp.sel(year=time_slice1).mean('year')
chill_hours_year_ssp2  = chill_hours_year_ssp.sel(year=time_slice2).mean('year')

In [ ]:
yearsum_ssp1 

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))
yearsum.plot(ax = ax, x='year',color = 'cornflowerblue', label='Historical model')
yearsum_ssp.plot(ax = ax, x='year', color = 'midnightblue', label='SSP585 model')
plt.title("Utah chill hour units - yearly count for the MPI-ESM1.2-HAM model")
ax.set_ylabel('hours')
ax.axhline(y=yearsum.mean('year'), color='lightsteelblue', label='Historical mean', xmin=0, xmax=0.46)
ax.axhline(yearsum_ssp1, color='black', label='SSP585 mean 2015-2035', xmin=0.46, xmax=0.72)
ax.axhline(yearsum_ssp2, color='black', label='SSP585 mean 2035-2055 ', xmin=0.72, xmax=1)
ax.axhline(2563, color='peachpuff', label='Historical mean chill hours', linewidth=3)
ax.legend(loc='upper right')
plt.show()

# Chill hour

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))
chill_hours_year.plot(ax = ax, x='year',color = 'cornflowerblue', label='Historical model')
chill_hours_year_ssp.plot(ax = ax, x='year', color = 'midnightblue', label='SSP585 model')
plt.title("Winter chill hours - yearly count for the MPI-ESM1.2-HAM model")
ax.set_ylabel('hours')
ax.axhline(y=chill_hours_year.mean('year'), color='lightsteelblue', label='Historical mean', xmin=0, xmax=0.46)
ax.axhline(chill_hours_year_ssp1, color='black', label='SSP585 mean 2015-2035', xmin=0.46, xmax=0.72)
ax.axhline(chill_hours_year_ssp2, color='black', label='SSP585 mean 2035-2055 ', xmin=0.72, xmax=1)
ax.axhline(2683, color='peachpuff', label='Historical mean chill hours', linewidth=3)
ax.legend(loc='upper right')
plt.show()

--------------------

---------------------

--------------------